In [1]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2

In [2]:
def is_multi_team_player(player,SwitchedTeamsList):
    return player in SwitchedTeamsList

def Clean_Mutli_Team_Players(df):
    
    #get list of players that have switched teams
    SwitchedTeamsList = df[df.Tm == 'TOT'].Player.unique()
    
    #only grab the TOT team rows for players that switched teams
    df['IsMultiTeamPlayer'] = df.Player.apply(lambda player: is_multi_team_player(player,SwitchedTeamsList))
    df=df[(df.Tm == 'TOT') | (df.IsMultiTeamPlayer == False)]
    df = df.drop(['Rk','IsMultiTeamPlayer'], axis=1)
    
    # Fill null values with zero
    for col in list(df.columns):
        df[col] = df[col].fillna(int(0))
    
    return df

In [3]:
Advanced2021 = Clean_Mutli_Team_Players(pd.read_csv("2021Advanced.csv"))
Totals2021 = Clean_Mutli_Team_Players(pd.read_csv("2021Totals.csv"))

In [4]:
# We are going to use the Advanced2021 df as the base and merge in some columns from the totals sheet

totalsColumnsToMerge =['Player','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA',
                       'FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GS']

#merge allNBA status into season stats
df_2021 = pd.merge(Advanced2021,Totals2021[totalsColumnsToMerge],how='left',left_on = ['Player'], right_on = ['Player'])

In [5]:
#for column names, replace % with _pct
for col in list(df_2021.columns):
    df_2021=df_2021.rename(columns = {col:col.replace('%','_pct')})


In [6]:
df_2021.head(10)

,Player,Pos,Age,Tm,G,MP,PER,TS_pct,3PAr,FTr,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GS
0,Precious Achiuwa,PF,21,MIA,61,737,14.2,0.550,0.004,0.482,...,73,135,208,29,20,28,43,91,304,4
1,Jaylen Adams,PG,24,MIL,7,18,-6.5,0.125,0.250,0.000,...,0,3,3,2,0,0,0,1,2,0
2,Steven Adams,C,27,NOP,58,1605,15.1,0.596,0.010,0.438,...,213,301,514,111,54,38,78,113,438,58
3,Bam Adebayo,C,23,MIA,64,2143,22.7,0.626,0.010,0.443,...,142,431,573,346,75,66,169,145,1197,64
4,LaMarcus Aldridge,C,35,TOT,26,674,15.7,0.556,0.270,0.159,...,19,99,118,49,11,29,27,47,352,23
5,Ty-Shon Alexander,SG,22,PHO,15,47,4.2,0.349,0.750,0.167,...,2,8,10,6,0,1,3,2,9,0
6,Nickeil Alexander-Walker,SG,22,NOP,46,1007,12.5,0.522,0.478,0.144,...,13,131,144,102,47,22,69,88,508,13
7,Grayson Allen,SG,25,MEM,50,1259,12.8,0.586,0.662,0.220,...,19,141,160,108,46,8,48,71,532,38
8,Jarrett Allen,C,22,TOT,63,1864,20.1,0.661,0.039,0.602,...,196,435,631,106,32,90,100,96,806,45
9,Al-Farouq Aminu,PF,30,TOT,23,434,8.9,0.469,0.374,0.222,...,22,88,110,31,19,9,28,29,102,14
